In [12]:
%pylab widget
%load_ext autoreload
%autoreload 2

from analysis_functions import *
import fastcat.fastcat as fc

from scipy.optimize import minimize

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# First lets look at the GOS

In [2]:
nviews = 360

s = fc.Spectrum()
s.load('Varian_truebeam_phasespace')
kernel = fc.Kernel(s,'CuGOS-784-micrometer')
angles = np.linspace(0,np.pi*2,nviews)        

phantom = fc.Catphan_404()
phantom.phan_map = ['air','CATPHAN_Polystyrene','CATPHAN_Acrylic','CATPHAN_B20','CATPHAN_Delrin_revised','water','CATPHAN_Teflon_revised','air','CATPHAN_PMP','CATPHAN_B50','CATPHAN_LDPE','water','CATPHAN_Polystyrene','air','CATPHAN_Acrylic'] 
phantom.return_projs(kernel,s,angles,det_on=True,scat_on=False,mgy=7000/nviews)

phantom.reconstruct('FDK',filt='ram_lak')

(19, 41, 41) 19
really doing it!
ratio is 26365.81808962475 number of photons 0.010763158668836276


In [13]:
# np.save('recons/6x_gos_recon_ram_lak_360',phantom.img[4])
slice_recon = np.load('recons/6x_gos_recon_ram_lak_360.npy')

In [14]:
plt.figure()
plt.imshow((slice_recon))

recon_slice = slice_recon
im = create_mask(recon_slice.shape,r=5.75)
contrast = []

for ii in range(2,int(np.max(im)+1)):
    
    contrast.append(np.mean(recon_slice[im == ii]))

# Get rid of the delrin
cc = np.array(sorted(contrast))[np.arange(len(contrast))]
star_data = np.array(star_data_gos[1::2])[np.arange(len(contrast))]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Now lets load the experimental data which was generated in ~/MV_CBCT and saved as a npy file

In [15]:
bcca_recon = np.load('/home/xcite/MV_CBCT/recon_6x_100.npy')

plt.figure()
plt.imshow(np.rot90(bcca_recon.T[45],2),vmax=0.012)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
recon_slice = np.rot90(bcca_recon.T[45],2)
im = create_mask(recon_slice.shape,off =[-0.2,0], r= 5.8)
contrast = []

for ii in range(2,int(np.max(im)+1)):
    
    contrast.append(np.mean(recon_slice[im == ii]))

# Get rid of the delrin
bcca_data = np.array(sorted(contrast))[np.arange(len(contrast))]

In [18]:
def HU_fun(a,x,y):
    return np.mean(np.abs((x - a[0])/(a[1])- y))

res = minimize(HU_fun,[cc[3],cc[0]],args=(cc,bcca_data))

x = [0,0,.853,0.945,.998,1.147,1.36,1.87]
plt.figure()
plt.plot(x,(cc - res.x[0])/(res.x[1] - res.x[0])*-1000,'s',c='cornflowerblue',label='fastCAT')
plt.plot(x,bcca_data,'s',c='darkorange',label='Star-lack')

fc_data = (cc - res.x[0])/(res.x[1] - res.x[0])*-1000

xx = np.linspace(0,max(x),100)
pp = np.polyfit(x,fc_data,1,full=1)
qq = np.polyfit(x,bcca_data,1,full=1)
yy = np.polyval(pp[0],xx)
zz = np.polyval(qq[0],xx)

plt.ylabel('HU')
plt.xlabel('Relative electron density')
plt.title('CWO')
plt.xticks(x,['air1','air2','PMP','LDPE','Poly','Acrylic','Delrin','Teflon'],rotation=75)
plt.plot(xx,yy,':',c='cornflowerblue',lw=0.8)
plt.plot(xx,zz,'-.',c='darkorange',lw = 0.8)
plt.legend()
plt.tight_layout()
plt.savefig('initial_cwo')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
plt.figure()
scale, offset, offset2 = 16.5, 0.00650, 0.057
plt.plot(scale*(bcca_data - offset),'rx')
plt.plot(cc - offset2,'kx')
plt.ylabel('Machine Output Value [??]')
plt.xlabel('Material')
plt.legend(['BCCA data','fastCAT'])
plt.title('Experimental Validation 6 MV 100MU')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Experimental Validation 6 MV 100MU')

In [153]:
1-pp[1]/np.std(fc_data)**2

array([0.99967224])

In [154]:
1-qq[1]/np.std(star_data)**2

array([1.])

In [ ]:
plt.figure()
recon_slice[im != 0] = 0
plt.imshow(recon_slice)

plt.figure()
recon_slice[im != 0] = 0
plt.imshow(recon_slice)

# Now lets look at the kV data 

In [181]:
plt.figure()
plt.imshow(kV_image[:,:,45])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
nviews = 360

In [6]:
s = fc.calculate_spectrum(100, 12, 3, 50)
s.attenuate(0.2,fc.get_mu(z=13))
kernel = fc.Kernel(s,'CsI-784-micrometer')
angles = np.linspace(0,np.pi*2,nviews)        

phantom = fc.Catphan_404()
phantom.phan_map = ['air','CATPHAN_Polystyrene','CATPHAN_Acrylic','CATPHAN_B20','CATPHAN_Delrin_revised','water','CATPHAN_Teflon_revised','air','CATPHAN_PMP','CATPHAN_B50','CATPHAN_LDPE','water','CATPHAN_Polystyrene','air','CATPHAN_Acrylic'] 
phantom.return_projs(kernel,s,angles,det_on=True,scat_on=False,mgy=7000/nviews)
phantom.reconstruct('FDK',filt='ram_lak')

Calculation:  1 / 50
Calculation:  2 / 50
Calculation:  3 / 50
Calculation:  4 / 50
Calculation:  5 / 50
Calculation:  6 / 50
Calculation:  7 / 50
Calculation:  8 / 50
Calculation:  9 / 50
Calculation:  10 / 50
Calculation:  11 / 50
Calculation:  12 / 50
Calculation:  13 / 50
Calculation:  14 / 50
Calculation:  15 / 50
Calculation:  16 / 50
Calculation:  17 / 50
Calculation:  18 / 50
Calculation:  19 / 50
Calculation:  20 / 50
Calculation:  21 / 50
Calculation:  22 / 50
Calculation:  23 / 50
Calculation:  24 / 50
Calculation:  25 / 50
Calculation:  26 / 50
Calculation:  27 / 50
Calculation:  28 / 50
Calculation:  29 / 50
Calculation:  30 / 50
Calculation:  31 / 50
Calculation:  32 / 50
Calculation:  33 / 50
Calculation:  34 / 50
Calculation:  35 / 50
Calculation:  36 / 50
Calculation:  37 / 50
Calculation:  38 / 50
Calculation:  39 / 50
Calculation:  40 / 50
Calculation:  41 / 50
Calculation:  42 / 50
Calculation:  43 / 50
Calculation:  44 / 50
Calculation:  45 / 50
Calculation:  46 / 

ValueError: operands could not be broadcast together with shapes (360,64,512) (512,18) 

In [282]:
# np.save('recons/kV_CsI_recon_ram_lak_360',phantom.img[4])
slice_recon = np.load('recons/kV_CsI_recon_ram_lak_360.npy')

In [283]:
plt.figure()
plt.imshow((slice_recon))

recon_slice = slice_recon
im = create_mask(recon_slice.shape,r=5.75)
contrast = []

for ii in range(2,int(np.max(im)+1)):
    
    contrast.append(np.mean(recon_slice[im == ii]))

# Get rid of the delrin
cc = np.array(sorted(contrast))[np.arange(len(contrast))]
star_data = np.array(star_data_gos[1::2])[np.arange(len(contrast))]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Now lets load the experimental kV data which was generated in ~/MV_CBCT and saved as a npy file

In [284]:
bcca_recon = np.load('/home/xcite/MV_CBCT/jericho_data/dicom_volume.npy')

plt.figure()
plt.imshow(np.rot90(bcca_recon.T[45],2))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [285]:
recon_slice = np.rot90(bcca_recon.T[45])
im = create_mask(recon_slice.shape,off = [-0.1,-0.02], r= 4.45, radius = 0.3)
contrast = []

for ii in range(2,int(np.max(im)+1)):
    
    contrast.append(np.mean(recon_slice[im == ii]))

# Get rid of the delrin
bcca_data = np.array(sorted(contrast))[np.arange(len(contrast))]

In [286]:
plt.figure()
recon_slice[im != 0] = 0
plt.imshow(recon_slice)

plt.figure()
recon_slice[im != 0] = 0
plt.imshow(recon_slice)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [288]:
def HU_fun(a,x,y):
    return np.mean(np.abs((x - a[0])/(a[1] - a[0])*-1000 - y))

res = minimize(HU_fun,[cc[3],cc[0]],args=(cc,bcca_data))

x = [0,0,.853,0.945,.998,1.147,1.36,1.87]
plt.figure()
plt.plot(x,(cc - res.x[0])/(res.x[1] - res.x[0])*-1000,'s',c='cornflowerblue',label='fastCAT')
plt.plot(x,bcca_data,'s',c='darkorange',label='Star-lack')

fc_data = (cc - res.x[0])/(res.x[1] - res.x[0])*-1000

xx = np.linspace(0,max(x),100)
pp = np.polyfit(x,fc_data,1,full=1)
qq = np.polyfit(x,bcca_data,1,full=1)
yy = np.polyval(pp[0],xx)
zz = np.polyval(qq[0],xx)

plt.ylabel('HU')
plt.xlabel('Relative electron density')
plt.title('CWO')
plt.xticks(x,['air1','air2','PMP','LDPE','Poly','Acrylic','Delrin','Teflon'],rotation=75)
plt.plot(xx,yy,':',c='cornflowerblue',lw=0.8)
plt.plot(xx,zz,'-.',c='darkorange',lw = 0.8)
plt.legend()
plt.tight_layout()
plt.savefig('initial_cwo')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [172]:
plt.figure()
scale, offset, offset2 = 16.5, 0.00650, 0.057
plt.plot(scale*(bcca_data - offset),'rx')
plt.plot(cc - offset2,'kx')
plt.xlabel('Machine Output Value [??]')
plt.ylabel('Material')
plt.legend(['BCCA data','fastCAT'])
plt.title('Experimental Validation 6 MV 100MU')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Experimental Validation 6 MV 100MU')

In [153]:
1-pp[1]/np.std(fc_data)**2

array([0.99967224])

In [154]:
1-qq[1]/np.std(star_data)**2

array([1.])